# 🎯 SkillBridge - Sistema de Recomendação de Cursos com IA

## Projeto: Global Solution - Artificial Intelligence & Chatbot

**Integrantes:**
- Bruno Vinicius Barbosa - RM566366
- João Pedro Bitencourt Goldoni - RM564339  
- Marina Tamagnini Magalhães - RM561786

**Instituição:** FIAP  
**Data:** Novembro 2025

---

## 📋 Objetivo

Desenvolver **2 modelos de Machine Learning** para recomendar cursos personalizados:

1. **Modelo de Regressão**: Prever score de relevância do curso (0-10)
2. **Modelo de Classificação**: Prever probabilidade de conclusão do curso

## 🎯 Contexto do Negócio

A plataforma SkillBridge possui:
- **173 cursos** em 15 carreiras diferentes
- Usuários com perfis variados (idade, experiência, tempo disponível)
- Objetivo: Recomendar cursos relevantes que o usuário tem chance de concluir

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from datetime import datetime

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Configurações
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Bibliotecas importadas com sucesso!")

---
## 📊 1. ANÁLISE EXPLORATÓRIA DOS DADOS
### 1.1 Carregamento dos Dados

In [ ]:
# Carregar dataset
df = pd.read_csv('dataset_treino.csv')

print("="*80)
print("INFORMAÇÕES DO DATASET")
print("="*80)
print(f"\nShape: {df.shape}")
print(f"Colunas: {df.shape[1]}")
print(f"Linhas: {df.shape[0]}")
print(f"\nPrimeiras linhas:")
df.head()

In [ ]:
# Informações do dataset
print("\nInfo:")
df.info()

print("\n" + "="*80)
print("ESTATÍSTICAS DESCRITIVAS")
print("="*80)
df.describe()

In [ ]:
# Verificar valores nulos
print("\nValores nulos:")
print(df.isnull().sum())
print(f"\n✓ Dataset sem valores nulos!")

### 1.2 Análise Visual dos Dados

In [ ]:
# Visualizações
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Score de Relevância
axes[0, 0].hist(df['score_relevancia'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribuição: Score de Relevância', fontsize=14, weight='bold')
axes[0, 0].set_xlabel('Score (0-10)')
axes[0, 0].set_ylabel('Frequência')

# Probabilidade de Conclusão
axes[0, 1].hist(df['probabilidade_conclusao'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('Distribuição: Probabilidade de Conclusão', fontsize=14, weight='bold')
axes[0, 1].set_xlabel('Probabilidade (0-1)')
axes[0, 1].set_ylabel('Frequência')

# Concluiu (Binário)
df['concluiu'].value_counts().plot(kind='bar', ax=axes[0, 2], color=['red', 'green'])
axes[0, 2].set_title('Conclusão de Cursos', fontsize=14, weight='bold')
axes[0, 2].set_xlabel('Concluiu')
axes[0, 2].set_ylabel('Quantidade')
axes[0, 2].set_xticklabels(['Não (0)', 'Sim (1)'], rotation=0)

# Idade
axes[1, 0].hist(df['idade'], bins=20, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_title('Distribuição: Idade dos Usuários', fontsize=14, weight='bold')
axes[1, 0].set_xlabel('Idade')
axes[1, 0].set_ylabel('Frequência')

# Anos de Experiência
axes[1, 1].hist(df['anos_experiencia'], bins=20, edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].set_title('Distribuição: Anos de Experiência', fontsize=14, weight='bold')
axes[1, 1].set_xlabel('Anos')
axes[1, 1].set_ylabel('Frequência')

# Tempo Disponível
axes[1, 2].hist(df['tempo_disponivel_semanal'], bins=15, edgecolor='black', alpha=0.7, color='cyan')
axes[1, 2].set_title('Distribuição: Tempo Disponível/Semana', fontsize=14, weight='bold')
axes[1, 2].set_xlabel('Horas')
axes[1, 2].set_ylabel('Frequência')

plt.tight_layout()
plt.savefig('analise_distribuicoes.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráficos de distribuição criados!")

In [ ]:
# Matriz de correlação
features_numericas = [
    'idade', 'anos_experiencia', 'tempo_disponivel_semanal',
    'nivel_experiencia_num', 'escolaridade_num', 'nivel_curso_num',
    'carga_horaria', 'avaliacao_media', 'taxa_conclusao_media',
    'popularidade_score', 'match_nivel', 'match_tempo', 'match_carreira',
    'score_relevancia', 'probabilidade_conclusao'
]

plt.figure(figsize=(16, 12))
correlation_matrix = df[features_numericas].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Features Numéricas', fontsize=16, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('matriz_correlacao.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Matriz de correlação criada!")

---
## 🔧 2. PREPARAÇÃO DOS DADOS

In [ ]:
# Definir features para cada modelo

# Features para REGRESSÃO (prever score_relevancia)
features_regressao = [
    'nivel_experiencia_num',
    'tempo_disponivel_semanal',
    'idade',
    'anos_experiencia',
    'escolaridade_num',
    'nivel_curso_num',
    'carga_horaria',
    'avaliacao_media',
    'taxa_conclusao_media',
    'popularidade_score',
    'match_nivel',
    'match_tempo',
    'match_carreira'
]

# Features para CLASSIFICAÇÃO (prever concluiu)
features_classificacao = [
    'nivel_experiencia_num',
    'tempo_disponivel_semanal',
    'idade',
    'anos_experiencia',
    'escolaridade_num',
    'nivel_curso_num',
    'carga_horaria',
    'avaliacao_media',
    'taxa_conclusao_media',
    'popularidade_score',
    'match_nivel',
    'match_tempo',
    'match_carreira',
    'progresso'  # Incluído para classificação
]

print("="*80)
print("FEATURES SELECIONADAS")
print("="*80)
print(f"\nRegressão ({len(features_regressao)} features):")
for f in features_regressao:
    print(f"  - {f}")

print(f"\nClassificação ({len(features_classificacao)} features):")
for f in features_classificacao:
    print(f"  - {f}")

# Salvar configuração de features
features_config = {
    'regressao': features_regressao,
    'classificacao': features_classificacao
}

with open('features.json', 'w') as f:
    json.dump(features_config, f, indent=2)

print("\n✓ Configuração de features salva em features.json")

In [ ]:
# Preparar dados para REGRESSÃO
X_reg = df[features_regressao]
y_reg = df['score_relevancia']

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print("="*80)
print("REGRESSÃO - SPLIT DOS DADOS")
print("="*80)
print(f"Treino: {X_reg_train.shape}")
print(f"Teste: {X_reg_test.shape}")

# Preparar dados para CLASSIFICAÇÃO
X_class = df[features_classificacao]
y_class = df['concluiu']

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

print("\n" + "="*80)
print("CLASSIFICAÇÃO - SPLIT DOS DADOS")
print("="*80)
print(f"Treino: {X_class_train.shape}")
print(f"Teste: {X_class_test.shape}")
print(f"\nDistribuição de classes no treino:")
print(y_class_train.value_counts())
print(f"\nDistribuição de classes no teste:")
print(y_class_test.value_counts())

---
## 🤖 3. MODELO 1 - REGRESSÃO (Random Forest)
### Objetivo: Prever score de relevância do curso (0-10)

In [ ]:
print("="*80)
print("TREINANDO MODELO DE REGRESSÃO")
print("="*80)

# Criar e treinar modelo
modelo_regressao = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

print("\nTreinando...")
modelo_regressao.fit(X_reg_train, y_reg_train)
print("✓ Modelo treinado!")

# Predições
y_reg_pred_train = modelo_regressao.predict(X_reg_train)
y_reg_pred_test = modelo_regressao.predict(X_reg_test)

# Métricas
rmse_train = np.sqrt(mean_squared_error(y_reg_train, y_reg_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_test))
r2_train = r2_score(y_reg_train, y_reg_pred_train)
r2_test = r2_score(y_reg_test, y_reg_pred_test)

print("\n" + "="*80)
print("RESULTADOS - REGRESSÃO")
print("="*80)
print(f"\nRMSE Treino: {rmse_train:.4f}")
print(f"RMSE Teste:  {rmse_test:.4f}")
print(f"\nR² Treino: {r2_train:.4f}")
print(f"R² Teste:  {r2_test:.4f}")

# Importância das features
feature_importance_reg = pd.DataFrame({
    'feature': features_regressao,
    'importance': modelo_regressao.feature_importances_
}).sort_values('importance', ascending=False)

print("\nImportância das Features (Top 10):")
print(feature_importance_reg.head(10))

In [ ]:
# Visualizar resultados da regressão
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Predito vs Real
axes[0].scatter(y_reg_test, y_reg_pred_test, alpha=0.5)
axes[0].plot([y_reg_test.min(), y_reg_test.max()], 
             [y_reg_test.min(), y_reg_test.max()], 
             'r--', lw=2)
axes[0].set_xlabel('Score Real', fontsize=12)
axes[0].set_ylabel('Score Predito', fontsize=12)
axes[0].set_title(f'Regressão: Predito vs Real\nR² = {r2_test:.4f}', 
                  fontsize=14, weight='bold')
axes[0].grid(True, alpha=0.3)

# Importância das features
feature_importance_reg.head(10).plot(kind='barh', x='feature', y='importance', 
                                      ax=axes[1], legend=False)
axes[1].set_xlabel('Importância', fontsize=12)
axes[1].set_title('Top 10 Features Mais Importantes', fontsize=14, weight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('resultado_regressao.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráficos de regressão criados!")

---
## 🎯 4. MODELO 2 - CLASSIFICAÇÃO (Random Forest)
### Objetivo: Prever se o usuário vai concluir o curso

In [ ]:
print("="*80)
print("TREINANDO MODELO DE CLASSIFICAÇÃO")
print("="*80)

# Criar e treinar modelo
modelo_classificacao = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

print("\nTreinando...")
modelo_classificacao.fit(X_class_train, y_class_train)
print("✓ Modelo treinado!")

# Predições
y_class_pred_train = modelo_classificacao.predict(X_class_train)
y_class_pred_test = modelo_classificacao.predict(X_class_test)
y_class_pred_proba_test = modelo_classificacao.predict_proba(X_class_test)[:, 1]

# Métricas
acc_train = accuracy_score(y_class_train, y_class_pred_train)
acc_test = accuracy_score(y_class_test, y_class_pred_test)
precision = precision_score(y_class_test, y_class_pred_test)
recall = recall_score(y_class_test, y_class_pred_test)
f1 = f1_score(y_class_test, y_class_pred_test)
roc_auc = roc_auc_score(y_class_test, y_class_pred_proba_test)

print("\n" + "="*80)
print("RESULTADOS - CLASSIFICAÇÃO")
print("="*80)
print(f"\nAcurácia Treino: {acc_train:.4f}")
print(f"Acurácia Teste:  {acc_test:.4f}")
print(f"\nPrecision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_class_test, y_class_pred_test, 
                            target_names=['Não Concluiu', 'Concluiu']))

# Importância das features
feature_importance_class = pd.DataFrame({
    'feature': features_classificacao,
    'importance': modelo_classificacao.feature_importances_
}).sort_values('importance', ascending=False)

print("\nImportância das Features (Top 10):")
print(feature_importance_class.head(10))

In [ ]:
# Visualizar resultados da classificação
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_class_test, y_class_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Não Concluiu', 'Concluiu'],
            yticklabels=['Não Concluiu', 'Concluiu'])
axes[0].set_title(f'Matriz de Confusão\nAcurácia = {acc_test:.4f}', 
                  fontsize=14, weight='bold')
axes[0].set_ylabel('Real', fontsize=12)
axes[0].set_xlabel('Predito', fontsize=12)

# Importância das features
feature_importance_class.head(10).plot(kind='barh', x='feature', y='importance', 
                                        ax=axes[1], legend=False, color='green')
axes[1].set_xlabel('Importância', fontsize=12)
axes[1].set_title('Top 10 Features Mais Importantes', fontsize=14, weight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('resultado_classificacao.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráficos de classificação criados!")

---
## 💾 5. SALVANDO MODELOS TREINADOS

In [ ]:
print("="*80)
print("SALVANDO MODELOS")
print("="*80)

# Salvar modelo de regressão
with open('modelo_regressao.pkl', 'wb') as f:
    pickle.dump(modelo_regressao, f)
print("\n✓ modelo_regressao.pkl salvo")

# Salvar modelo de classificação
with open('modelo_classificacao.pkl', 'wb') as f:
    pickle.dump(modelo_classificacao, f)
print("✓ modelo_classificacao.pkl salvo")

# Salvar métricas
metricas = {
    'regressao': {
        'rmse_train': float(rmse_train),
        'rmse_test': float(rmse_test),
        'r2_train': float(r2_train),
        'r2_test': float(r2_test)
    },
    'classificacao': {
        'accuracy_train': float(acc_train),
        'accuracy_test': float(acc_test),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'roc_auc': float(roc_auc)
    },
    'data_treinamento': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('metricas_modelos.json', 'w') as f:
    json.dump(metricas, f, indent=2)
print("✓ metricas_modelos.json salvo")

print("\n" + "="*80)
print("✅ TREINAMENTO CONCLUÍDO COM SUCESSO!")
print("="*80)
print("\nArquivos gerados:")
print("  • modelo_regressao.pkl")
print("  • modelo_classificacao.pkl")
print("  • features.json")
print("  • metricas_modelos.json")
print("\nPróximo passo: Criar API Flask para servir os modelos")

---
## 📊 6. RESUMO DOS RESULTADOS

### Modelo de Regressão
- **Objetivo:** Prever score de relevância (0-10)
- **Algoritmo:** Random Forest Regressor
- **R² Test:** ~0.95 (excelente!)
- **RMSE Test:** ~0.5 pontos

### Modelo de Classificação
- **Objetivo:** Prever se o usuário vai concluir
- **Algoritmo:** Random Forest Classifier
- **Acurácia Test:** ~95%
- **F1-Score:** ~95%
- **ROC-AUC:** ~98%

### Features Mais Importantes
1. **match_carreira** - Match entre curso e carreira desejada
2. **progresso** - Progresso atual do usuário (para classificação)
3. **match_nivel** - Compatibilidade de nível
4. **taxa_conclusao_media** - Taxa histórica de conclusão
5. **popularidade_score** - Popularidade do curso

### Conclusão
Os modelos apresentaram **excelente performance** e estão prontos para uso em produção via API Flask!